In [2]:
import numpy as np
import pandas as pd

k = 2
df = pd.read_csv('salario.csv')

# removendo atributos sensíveis
df.drop('id', axis=1, inplace=True)
df.drop('salario', axis=1, inplace=True)
# adicionando coluna localização
df.insert(0, 'localizacao', '*')

freq_pais = df["pais"].value_counts()
freq_estado = df["estado"].value_counts()
freq_cidade = df["cidade"].value_counts()
freq_data = df["data"].value_counts()

# generalizando localização
for i in df.index:
    c = df.at[i, 'cidade']
    e = df.at[i, 'estado']
    p = df.at[i, 'pais']
    if (freq_cidade.loc[c] < k):
        df.at[i, 'cidade'] = '*'
        if(freq_estado.loc[e] < k):
            df.at[i, 'estado'] = '*'
            if(freq_pais.loc[p] < k):
                df.at[i, 'localizacao'] = '*'
            else: df.at[i, 'localizacao'] = p
        else: df.at[i, 'localizacao'] = e + ', ' + p
    else: df.at[i, 'localizacao'] = c + ', ' + e + ', ' + p
        
#removendo colunas extras: cidade, estado e país
df.drop('cidade', axis=1, inplace=True)
df.drop('estado', axis=1, inplace=True)
df.drop('pais', axis=1, inplace=True)

# verificando classes de equivalência para data: suprimindo dia
for i in df.index:
    d = df.at[i, 'data']
    aux = d.split('/')
    if (freq_data.loc[d] < k):
        aux[0] = '**'
    new_aux = aux[0] + '/' + aux[1] + '/' + aux[2]
    df.at[i, 'data'] = new_aux

freq_data = df["data"].value_counts()
# verificando classes de equivalência para data: suprimindo mês
for i in df.index:
    d = df.at[i, 'data']
    aux = d.split('/')
    if (freq_data.loc[d] < k):
        aux[1] = '**'
    new_aux = aux[0] + '/' + aux[1] + '/' + aux[2]
    df.at[i, 'data'] = new_aux
    
freq_data = df["data"].value_counts()
# verificando classes de equivalência para data: suprimindo ano
for i in df.index:
    d = df.at[i, 'data']
    aux = d.split('/')
    if (freq_data.loc[d] < k):
        aux[2] = '****'
    new_aux = aux[0] + '/' + aux[1] + '/' + aux[2]
    df.at[i, 'data'] = new_aux
    
freq_data = df["data"].value_counts


export_csv = df.to_csv(r'salario_2.csv', index=None, header=True)


FileNotFoundError: [Errno 2] File b'salario.csv' does not exist: b'salario.csv'